In [5]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper , ArxivAPIWrapper
from langchain_community.utilities import GoogleFinanceAPIWrapper
from langchain_community.tools.google_finance import GoogleFinanceQueryRun

In [ ]:
#inbuilt tool of wikipedia
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [6]:
api_wrapper_arrxiv = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=300)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arrxiv)
arxiv.name

'arxiv'

In [ ]:
tools=[wiki,arxiv]

In [7]:
#Custom tool [RAG Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [12]:
loader = WebBaseLoader("https://python.langchain.com/api_reference/reference.html#integrations")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents,HuggingFaceEmbeddings())
retriever = vectordb.as_retriever()
retriever


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001F0B3BCD810>, search_kwargs={})

In [13]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"langsmith-search","Search any information about Langsmith")
retriever_tool.name

'langsmith-search'

In [14]:
tools=[wiki,arxiv,retriever_tool]

In [20]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq
import os 
load_dotenv() 
groq_api_key = os.getenv('GROQ_API_KEY')
langchain_api_key = os.getenv('LANGCHAIN_API_KEY')

llm = ChatGroq(groq_api_key=groq_api_key,model="Llama3-8b-8192")

In [17]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F0B4791C50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F0B4410DD0>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [28]:
from langchain import hub
from langchain_openai import ChatOpenAI

In [29]:
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [30]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [31]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [34]:
agent_executor.invoke({"input":"Tell me about Tanjiro Kamado"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Tanjiro Kamado'}`


Page: Tanjiro Kamado
Summary: Tanjiro Kamado (Japanese: 竈門 炭治郎, Hepburn: Kamado Tanjirō) is a fictional character and the protagonist of Koyoharu Gotouge's manga Demon Slayer: Kimetsu no Yaiba. Tanjiro goes on a quest to restore the humanity of his s
Invoking: `wikipedia` with `{'query': 'Tanjiro Kamado'}`


Page: Tanjiro Kamado
Summary: Tanjiro Kamado (Japanese: 竈門 炭治郎, Hepburn: Kamado Tanjirō) is a fictional character and the protagonist of Koyoharu Gotouge's manga Demon Slayer: Kimetsu no Yaiba. Tanjiro goes on a quest to restore the humanity of his sIt appears that the tool call "call_265g" yielded the same information as the previous tool call "call_ew06". In this case, I can provide a more detailed summary about Tanjiro Kamado.

Tanjiro Kamado is the main protagonist of the Demon Slayer: Kimetsu no Yaiba series. He is a kind and determined young boy who lives with his family in the

{'input': 'Tell me about Tanjiro Kamado',
 'output': 'It appears that the tool call "call_265g" yielded the same information as the previous tool call "call_ew06". In this case, I can provide a more detailed summary about Tanjiro Kamado.\n\nTanjiro Kamado is the main protagonist of the Demon Slayer: Kimetsu no Yaiba series. He is a kind and determined young boy who lives with his family in the mountains. After his family is slaughtered by demons, Tanjiro sets out on a quest to find a cure for his sister Nezuko, who has been turned into a demon. Along the way, he meets other characters who join him on his journey and together they face various challenges and battles against demons.\n\nTanjiro\'s character is known for his kindness, compassion, and strong sense of justice. He is also a skilled fighter and uses his unique breathing technique, the "Breathing Technique of the Sun", to fight against demons.\n\nWould you like to know more about Tanjiro Kamado or his adventures in the Demon Sl